# 문장 수준 임베딩
- 잠재 의미 분석(LSA) : 행렬분해
- 잠재 디리클레 할당(LDA) : 확률모델, Doc2Vecm ELMo, GPT, BERT
- 특히, GPT, BERT등은 Self Attention기반의 트랜스포머 네트워크로 구성됨

<b><h3>1. 잠재 의미 분석</h3></b><br> 
- 잠재 의미 분석 : 단어-문서 행렬이나 TF-IDF 행렬, 단어-문맥 행렬 또는 PMI행렬에 특이값 분해로 차원 축소를 시행하고 단어에 해당하는 벡터를 취해 임베딩
- 

<b><h3>3. 잠재 디리클레 할당(LDA)</h3></b><br> 

![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/lda%EB%8F%84%EC%8B%9D.png?raw=True)
- LDA는 토픽별 단어의 분포, 문서별 토픽의 분포를 추정
- 특정 토픽에 특정 단어가 나타날 확률, 위 도식에서 노란색 토픽엔 gene이라는 단어가 등장확률이 0.04, dna는 0.02, genetic은 0.01로 노란색 토픽은 대략 "유전자"라는 주제 예상
- 주어진 문서를 보면 노란색 토픽에 해당하는 단어가 많다. 따라서 문서의 메인 주제는 노란색 토픽일 가능성이 높다.
- LDA는 글쓰기 예를 자주 든다.
- 글을 쓰기 전에 (1) 주제를 정한다. (2) 실제 글을 쓸 때 어떤 단어를 쓸지 결정한다.
- LDA는 (1) 말뭉치로부터 얻은 토픽 분포로부터 토픽을 뽑는다. (2) 해당 토픽에 해당하는 단어를 뽑는다.
- LDA는 말뭉치 이면에 존재하는 정보를 추론해기에 잠재라는 이름을 붙인것이다.

![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA1.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA2.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA3.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA4.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA5.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA6.jpg?raw=True)

- 튜토리얼
<LDA 모델 피처 생성> 
    - 문서 내 단어의 등장 순서를 고려하지 않고 단어 카운팅(빈도)만 체크

In [16]:
from gensim import corpora
from preprocess import get_tokenizer

corpus_fname = "C:/Users/fxk/PycharmProjects/tenjumh/NLP/KOREAN EMBEDDING/corrected_ratings_corpus.txt"

documents, tokenized_corpus = [], []
tokenizer = get_tokenizer("kkma")

with open(corpus_fname, 'r', encoding='utf-8') as f:
    for document in f:
        tokens = list(set(tokenizer.morphs(document.strip())))
        documents.append(document)
        tokenized_corpus.append(tokens)
dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space

![TF-IDF행렬](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA-3
.jpg?raw=True)